In [12]:
from glob import glob
from humemai.utils import read_yaml
import pandas as pd
import numpy as np


def determine_hyper_parameters(train: dict) -> dict:
    """Determine hyper parameters."""
    hyper_parameters = {}

    if "capacity" in train:
        hyper_parameters["capacity"] = (
            train["capacity"]["episodic"] + train["capacity"]["semantic"]
        )
        if train["capacity"]["episodic"] == 0:
            hyper_parameters["agent_type"] = "semantic"
        elif train["capacity"]["semantic"] == 0:
            hyper_parameters["agent_type"] = "episodic"
        elif train["capacity"]["episodic"] > 0 and train["capacity"]["semantic"] > 0:
            hyper_parameters["agent_type"] = "hybrid"
        else:
            raise ValueError("Unknown agent type")
    else:
        hyper_parameters["history_block_size"] = int(train["history_block_size"])
        hyper_parameters["agent_type"] = "baseline"

    hyper_parameters["room_size"] = train["env_config"]["room_size"]

    # hyper_parameters["train_seed"] = train["train_seed"]
    # hyper_parameters["test_seed"] = train["test_seed"]

    return hyper_parameters


def nanmean(data):
    return None if np.isnan(data).any() else round(np.mean(data))


def nanstd(data):
    return None if np.isnan(data).any() else round(np.std(data))


paths = glob("training-results/baselines/*/*/*/results.yaml") + glob(
    "training-results/dqn/*/*/*/*/explore/results.yaml"
)
final = {}
for path in paths:
    result_explore = read_yaml(path)
    val_score_explore = max([foo["mean"] for foo in result_explore["validation_score"]])
    test_score_explore = result_explore["test_score"]["mean"]

    if "baseline" in path:
        val_score_mm = np.nan
        test_score_mm = np.nan
    else:
        result_mm = read_yaml(path.replace("explore/results.yaml", "mm/results.yaml"))
        val_score_mm = max([foo["mean"] for foo in result_mm["validation_score"]])
        test_score_mm = result_mm["test_score"]["mean"]

    if "baseline" in path:
        train = read_yaml(path.replace("results.yaml", "train.yaml"))
    else:
        train = read_yaml(path.replace("explore/results.yaml", "train.yaml"))

    hp = determine_hyper_parameters(train)

    hp_str = str(hp)
    if hp_str in final:
        final[hp_str]["val_explore"].append(val_score_explore)
        final[hp_str]["test_explore"].append(test_score_explore)

        final[hp_str]["val_mm"].append(val_score_mm)
        final[hp_str]["test_mm"].append(test_score_mm)
    else:
        final[hp_str] = {
            "val_explore": [val_score_explore],
            "test_explore": [test_score_explore],
            "val_mm": [val_score_mm],
            "test_mm": [test_score_mm],
            "hyper_parameters": hp,
        }

df_list = []
for key in final:
    data = final[key]
    hp = data["hyper_parameters"]
    df_list.append(
        {
            "test": nanmean(data["test_explore"]),
            "std_test": nanstd(data["test_explore"]),
            "val": nanmean(data["val_explore"]),
            "std_val": nanstd(data["val_explore"]),
            "test_mm": nanmean(data["test_mm"]),
            "std_test_mm": nanstd(data["test_mm"]),
            "val_mm": nanmean(data["val_mm"]),
            "std_val_mm": nanstd(data["val_mm"]),
            "num_runs": len(data["test_explore"]),
            "capacity": hp.get("capacity", None),
            "agent_type": hp.get("agent_type", None),
            "room_size": hp.get("room_size", None),
            "history_block_size": hp.get("history_block_size", None),
        }
    )

df = pd.DataFrame(df_list)
df_sorted = df.sort_values(
    by=["history_block_size", "capacity", "test"], ascending=[True, True, False]
)
df_sorted

,test,std_test,val,std_val,test_mm,std_test_mm,val_mm,std_val_mm,num_runs,capacity,agent_type,room_size,history_block_size
0,215,18,250,8,None,None,None,None,5,None,baseline,l,1
2,240,34,266,8,None,None,None,None,5,None,baseline,l,2
5,214,19,259,13,None,None,None,None,5,None,baseline,l,4
4,311,71,322,63,None,None,None,None,5,None,baseline,l,8
3,353,107,384,136,None,None,None,None,5,None,baseline,l,16
1,296,43,330,38,None,None,None,None,3,None,baseline,l,32
